In [1]:
import os

from skimage import io, transform, color
import numpy as np

In [2]:
def resize_imgs(dir_path, op_dir_path, size, padding=255):
    '''
    Resize all images present in the given directory
    '''
    files = os.listdir(dir_path)
    for file_name in files:
        file_path = os.path.join(dir_path, file_name)
        if os.path.splitext(file_name)[1] not in ['.jpg', '.JPG', '.png', '.PNG']:
            continue
        
        # Read the image in Gray scale
        img = io.imread(file_path)
        img = color.rgb2gray(img)
        img = np.asarray(img * 255, np.uint8)
        reszd_img = resize_image(img, size)
        
        op_file_path = os.path.join(op_dir_path, 'reszd_' + file_name)
        io.imsave(op_file_path, reszd_img)
    return True

In [3]:
def resize_image(img, size, padding=255):
    '''
    Resize the given image to specified size
    '''
    # Append zeros or ones based on the choice of padding
    # to maintain the aspect ratio of the image
    rows, cols = img.shape[:2]
    max_size = max(rows, cols)
    canvas = np.ones((max_size, max_size), np.uint8) * padding
    
    # Place the actual image at the center of the canvas
    c_x, c_y = max_size / 2, max_size / 2
    x_beg = max(0, c_x - (cols / 2))
    y_beg = max(0, c_y - (rows / 2))
    x_end, y_end = x_beg + cols, y_beg + rows
    canvas[y_beg: y_end, x_beg: x_end] = img
        
    # Resize the image and write to op_dir_path specified
    reszd_img = transform.resize(canvas, (size, size))
    reszd_img = np.asarray(reszd_img * 255, np.uint8)
    return reszd_img

In [14]:
resize_imgs('../data/nuts_n_bolts_master/all/train/nuts/', '../data/nuts_n_bolts_master/all/50x50/train/nuts/', 50)

True

In [4]:
def imgs_to_csv(img_dirs, csv_path, labels):
    '''
    Writes image data into a CSV file for training
    Input:
        img_dirs = List of image directories for different class images
        csv_path = Ouput CSV file path
        labels = List of lables corresponding to the img_dirs
    '''
    X_list, y_list = [], []
    for img_dir, label in zip(img_dirs, labels):
        files = os.listdir(img_dir)
        for file_name in files:
            if os.path.splitext(file_name)[1] not in ['.jpg', '.JPG', '.png', '.PNG']:
                continue
            file_path = os.path.join(img_dir, file_name)
            img = io.imread(file_path, as_gray=True)
            X_list.append(img.flatten())
            y_list.append([label])
    X = np.array(X_list, np.float) / 255
    y = np.array(y_list, np.float)
    train_data = np.append(X, y, axis=1)
    np.savetxt(csv_path, train_data, delimiter=',', fmt='%f')

In [7]:
img_dirs = ['../data/nuts_n_bolts_master/all_svm/50x50/train/nuts/',
            '../data/nuts_n_bolts_master/all_svm/50x50/train/bolts/']
labels = [0, 1]
csv_path = '../data/nuts_n_bolts_master/all_svm/50x50/nuts_and_bolts_train.csv'

In [8]:
imgs_to_csv(img_dirs, csv_path, labels)